In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import LinearSVC 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

In [2]:
dados_ = pd.read_csv('train.csv')
dados_teste_ = pd.read_csv('test.csv')
display(dados_.head())
display(dados_teste_.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [51]:
dados_teste_.dropna(axis=1,inplace=True)
dados = dados_.drop('Name',axis=1)
dados_novo = dados_teste_.drop('Name',axis=1)
display(dados.info())
display(dados.columns)
display(dados_novo)
display(dados)
dados_novo.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


None

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

,PassengerId,Pclass,Sex,SibSp,Parch,Ticket,Embarked
0,892,3,male,0,0,330911,Q
1,893,3,female,1,0,363272,S
2,894,2,male,0,0,240276,Q
3,895,3,male,0,0,315154,S
4,896,3,female,1,1,3101298,S
...,...,...,...,...,...,...,...
413,1305,3,male,0,0,A.5. 3236,S
414,1306,1,female,0,0,PC 17758,C
415,1307,3,male,0,0,SOTON/O.Q. 3101262,S
416,1308,3,male,0,0,359309,S


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C


(418, 7)

In [25]:
minmax = MinMaxScaler()
onehot = OneHotEncoder(sparse=False,drop='first')

In [5]:
def criadorxy(dados,input_colunas,output_colunas):
    dados.dropna(axis=0,inplace=True)
    x = dados[input_colunas].copy()
    x_numero = x.select_dtypes(exclude='object')
    x_numero = minmax.fit_transform(x_numero)
    x_numero = pd.DataFrame(data=x_numero,columns=x.select_dtypes(exclude='object').columns)
    x_cat = x.select_dtypes(include='object')
    x_cat = onehot.fit_transform(x_cat)
    colunas = []
    for c,values in zip(x.select_dtypes(include='object').columns,onehot.categories_):
        for value in values[1:]:
            colunas.append(f'{c}_{value}')
    x_cat = pd.DataFrame(data=x_cat,columns=colunas)
    x = x_numero.join(x_cat)
    y = dados[output_colunas].copy()
    return x,y
x,y = criadorxy(dados,['PassengerId','Pclass','Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked',],'Survived')
print(f'Array = dtype {y.dtype}, shape{y.shape}, size{y.size}')

Array = dtype int64, shape(183,), size183


/home/gustavohenrique06112005/.local/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [57]:
x_treino,x_teste,y_treino,y_teste = train_test_split(x,y,train_size=2/3,random_state=1) 
y_treino = np.ravel(y_treino)
cerebro = LinearSVC()
cerebro.fit(x_treino,y_treino)
accuracy_score(y_teste,cerebro.predict(x_teste))

0.8360655737704918

In [65]:
x = dados_novo.copy()
x_min = x.select_dtypes(include='object')
x_min = onehot.fit_transform(x_min)
x_bin = x.select_dtypes(exclude='object')
x_bin = minmax.fit_transform(x_bin)
x_novo = np.append(x_bin,x_min,axis=1)

/home/gustavohenrique06112005/.local/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
